**Python supports multiple creational, structural, and behavioral design patterns.**
Design Pattern Summary Table
Pattern     Category    Use Case
Singleton	Creational	Logging, Configs, Database Connections
Factory	    Creational	Object Creation without Specifying Class
Adapter	    Structural	Making Legacy Code Work with New Systems
Decorator	Structural	Adding Functionality Without Modifying Code
Observer	Behavioral	Event-Driven Systems, Notifications
Strategy	Behavioral	Switching Algorithms at Runtime

✔ Need a single shared instance? → Singleton
✔ Want flexible object creation? → Factory
✔ Need to integrate old and new code? → Adapter
✔ Want to add behavior dynamically? → Decorator
✔ Need event notifications? → Observer
✔ Want flexible behavior switching? → Strategy

1. Creational Design Patterns (How objects are created and managed.)
🔹 Singleton Pattern (Ensures only one instance of a class exists.)
✅ Best for: Logging, configuration settings, database connections.

In [1]:
class Singleton:
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(Singleton, cls).__new__(cls)
        return cls._instance

obj1 = Singleton()
obj2 = Singleton()
print(obj1 is obj2)  # Output: True (Same instance)

True


🔹 Factory Pattern (Creates objects dynamically without specifying exact class.)
✅ Best for: Object creation with different configurations.

In [2]:
class Dog:
    def speak(self):
        return "Woof!"

class Cat:
    def speak(self):
        return "Meow!"

def animal_factory(animal_type):
    animals = {"dog": Dog, "cat": Cat}
    return animals.get(animal_type, Dog)()  

animal = animal_factory("cat")
print(animal.speak())  # Output: Meow!

Meow!


In [ ]:
# Second Example for Factory Pattern
from abc import ABC, abstractmethod
import cx_Oracle
import pymongo
import pandas as pd

# Step 1: Abstract Base Class
class DatabaseConnector(ABC):
    @abstractmethod
    def connect(self):
        pass

    @abstractmethod
    def fetch_data(self, query):
        pass

    @abstractmethod
    def insert_data(self, data):
        pass

# Step 2: Oracle Database Connector
class OracleConnector(DatabaseConnector):
    def __init__(self, dsn, user, password):
        self.dsn = dsn
        self.user = user
        self.password = password
        self.connection = None

    def connect(self):
        self.connection = cx_Oracle.connect(self.user, self.password, self.dsn)
        print("Connected to Oracle")

    def fetch_data(self, query):
        if not self.connection:
            raise Exception("Oracle connection not established")
        cursor = self.connection.cursor()
        cursor.execute(query)
        columns = [col[0] for col in cursor.description]  # Get column names
        data = cursor.fetchall()
        cursor.close()
        return pd.DataFrame(data, columns=columns)  # Store data in Pandas DataFrame

    def insert_data(self, data):
        raise NotImplementedError("Insert operation is not implemented for Oracle.")

# Step 3: MongoDB Database Connector
class MongoDBConnector(DatabaseConnector):
    def __init__(self, uri, database, collection):
        self.uri = uri
        self.database = database
        self.collection = collection
        self.client = None
        self.db = None
        self.collection_ref = None

    def connect(self):
        self.client = pymongo.MongoClient(self.uri)
        self.db = self.client[self.database]
        self.collection_ref = self.db[self.collection]
        print("Connected to MongoDB")

    def fetch_data(self, query):
        return list(self.collection_ref.find(query))

    def insert_data(self, data):
        if not self.collection_ref:
            raise Exception("MongoDB connection not established")
        if isinstance(data, pd.DataFrame):
            data = data.to_dict(orient="records")  # Convert DataFrame to a list of dictionaries
        self.collection_ref.insert_many(data)
        print("Data inserted into MongoDB successfully")

# Step 4: Factory Class
class DatabaseFactory:
    @staticmethod
    def get_connector(db_type, **kwargs):
        if db_type == "oracle":
            return OracleConnector(kwargs["dsn"], kwargs["user"], kwargs["password"])
        elif db_type == "mongodb":
            return MongoDBConnector(kwargs["uri"], kwargs["database"], kwargs["collection"])
        else:
            raise ValueError("Unsupported database type")

# Step 5: Fetch data from Oracle and insert into MongoDB
if __name__ == "__main__":
    # Oracle Configuration
    oracle_config = {
        "dsn": "localhost:1521/XEPDB1",
        "user": "my_user",
        "password": "my_password"
    }
    
    # MongoDB Configuration
    mongo_config = {
        "uri": "mongodb://localhost:27017/",
        "database": "company",
        "collection": "employees"
    }

    # Step 5.1: Get Oracle Connector and Fetch Data
    oracle_db = DatabaseFactory.get_connector("oracle", **oracle_config)
    oracle_db.connect()
    query = "SELECT * FROM employees"
    oracle_data = oracle_db.fetch_data(query)

    print("Fetched Data from Oracle:")
    print(oracle_data.head())  # Display first few rows

    # Step 5.2: Get MongoDB Connector and Insert Data
    mongo_db = DatabaseFactory.get_connector("mongodb", **mongo_config)
    mongo_db.connect()
    mongo_db.insert_data(oracle_data)

    print("Data transfer from Oracle to MongoDB completed successfully!")


2. Structural Design Patterns 🏛 (How objects are composed to form larger structures.)
🔹 Adapter Pattern (Allows incompatible interfaces to work together.)
✅ Best for: Making legacy code work with new systems.

In [3]:
class EuropeanPlug:
    def plug_type(self):
        return "European Plug"

class Adapter:
    def __init__(self, plug):
        self.plug = plug

    def plug_type(self):
        return f"Adapter -> {self.plug.plug_type()}"

plug = EuropeanPlug()
adapter = Adapter(plug)
print(adapter.plug_type())  # Output: Adapter -> European Plug

Adapter -> European Plug


In [ ]:
# Second Example For Adapter Pattern:
from abc import ABC, abstractmethod

# Step 1: Target Interface (Unified Method)
class DataSource(ABC):
    @abstractmethod
    def read_data(self):
        pass

# Step 2: Existing Incompatible Classes
class OldDatabase:
    """Legacy Database System"""
    def get_data(self):
        return "Data from Old Database"

class NewAPI:
    """Modern API System"""
    def fetch_data(self):
        return "Data from New API"

# Step 3: Adapter Classes
class DatabaseAdapter(DataSource):
    """Adapter for OldDatabase"""
    def __init__(self, old_db):
        self.old_db = old_db

    def read_data(self):
        return self.old_db.get_data()  # Adapting to new method

class APIAdapter(DataSource):
    """Adapter for NewAPI"""
    def __init__(self, new_api):
        self.new_api = new_api

    def read_data(self):
        return self.new_api.fetch_data()  # Adapting to new method

# Step 4: Client Code (Works with Unified Interface)
def fetch_from_source(source: DataSource):
    print(source.read_data())

# Usage Example
old_db = OldDatabase()
api = NewAPI()

# Wrapping incompatible classes with Adapters
db_adapter = DatabaseAdapter(old_db)
api_adapter = APIAdapter(api)

# Client code works with both through the common interface
fetch_from_source(db_adapter)  # Output: Data from Old Database
fetch_from_source(api_adapter)  # Output: Data from New API


🔹 Decorator Pattern (Dynamically adds behavior to objects without modifying their structure.)
✅ Best for: Logging, authentication, data transformation.

In [4]:
def uppercase_decorator(func):
    def wrapper():
        return func().upper()
    return wrapper

@uppercase_decorator
def greet():
    return "hello"

print(greet())  # Output: HELLO


HELLO


3. Behavioral Design Patterns 🎭 (How objects communicate and interact.)
🔹 Observer Pattern (Notifies multiple objects when a state changes.)
✅ Best for: Event-driven programming (e.g., UI updates, pub-sub systems).

In [5]:
class Observer:
    def update(self, message):
        print(f"Received update: {message}")

class Subject:
    def __init__(self):
        self.observers = []

    def add_observer(self, observer):
        self.observers.append(observer)

    def notify(self, message):
        for observer in self.observers:
            observer.update(message)

subject = Subject()
observer1 = Observer()
observer2 = Observer()

subject.add_observer(observer1)
subject.add_observer(observer2)
subject.notify("New Event!")


Received update: New Event!
Received update: New Event!


🔹 Strategy Pattern (Allows switching algorithms dynamically.)
✅ Best for: Choosing different behaviors at runtime.

In [6]:
class PayPal:
    def pay(self, amount):
        return f"Paid {amount} using PayPal"

class CreditCard:
    def pay(self, amount):
        return f"Paid {amount} using Credit Card"

class PaymentContext:
    def __init__(self, strategy):
        self.strategy = strategy

    def execute_payment(self, amount):
        return self.strategy.pay(amount)

payment = PaymentContext(CreditCard())
print(payment.execute_payment(100))  # Output: Paid 100 using Credit Card


Paid 100 using Credit Card
